In [1]:
import mido
import os
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
import numpy as np

Using TensorFlow backend.
/Users/StephanieBrito/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [2]:
mid = MidiFile('Data/Bamboleo.mid')  
test_mid = MidiFile('Data/ahoraquien.mid')

In [3]:
notes = []
test_notes = []

def isPercussion(channel):
    return channel in range(8, 17) or channel in range(35, 82)

for msg in mid:
#     if msg.type == 'program_change':
#         print(msg)
    if not msg.is_meta and msg.channel == 0 and msg.type == 'note_on':
        data = msg.bytes()
        notes.append(data[1])
        
        
for msg in test_mid:
#     if msg.type == 'program_change':
#         print(msg)
    if not msg.is_meta and msg.channel == 0 and msg.type == 'note_on':
        data = msg.bytes()
        test_notes.append(data[1])

In [4]:
# subsample data for training and prediction
X = []
y = []
# number of notes in a batch
n_prev = 4
for i in range(len(notes)-n_prev):
    X.append(notes[i:i+n_prev])
    y.append(notes[i+n_prev])
# save a seed to do prediction later
X_test = X[-300:]
X = np.asarray(X[:-300])
y = y[:-300]


In [5]:
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
print(len(dummy_y[0]))

24


In [10]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras import optimizers

# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(64, input_dim=n_prev, activation='relu'))
#     model.add(Dense(124, activation='relu'))
#     model.add(Dense(64, activation='relu'))
    model.add(Dense(len(dummy_y[0]), activation='softmax'))
    # Compile model
    sgd = optimizers.SGD(lr=0.000001, decay=1e-6, momentum=0.9, nesterov=True)

    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

In [11]:
model = baseline_model()
ll = model.fit(X, dummy_y, 32, 50, verbose=1)

Epoch 1/50
3806/3806 [==============================] - 0s 89us/step - loss: 14.4010 - acc: 0.0284
Epoch 2/50
3806/3806 [==============================] - 0s 47us/step - loss: 13.8555 - acc: 0.0297
Epoch 3/50
3806/3806 [==============================] - 0s 43us/step - loss: 13.1258 - acc: 0.0444
Epoch 4/50
3806/3806 [==============================] - 0s 58us/step - loss: 12.3918 - acc: 0.0596
Epoch 5/50
3806/3806 [==============================] - 0s 60us/step - loss: 11.5235 - acc: 0.0738
Epoch 6/50
3806/3806 [==============================] - 0s 48us/step - loss: 10.9164 - acc: 0.0851
Epoch 7/50
3806/3806 [==============================] - 0s 41us/step - loss: 10.5111 - acc: 0.0846
Epoch 8/50
3806/3806 [==============================] - 0s 42us/step - loss: 10.3876 - acc: 0.0975
Epoch 9/50
3806/3806 [==============================] - 0s 45us/step - loss: 10.3673 - acc: 0.0983
Epoch 10/50
3806/3806 [==============================] - 0s 40us/step - loss: 10.3555 - acc: 0.0998
Epoch 11/

In [13]:
X_test = []
for i in range(len(test_notes)-n_prev):
    X_test.append(test_notes[i:i+n_prev])
    

prediction = model.predict(np.array(X_test))
print(prediction[0])

class_labels = np.argmax(prediction, axis=1)
labels = list(set(y))
final_labels = [labels[i] for i in class_labels]
print(set(final_labels))
print(set(test_notes))
# for i in c
# print(list(set(y)))
# print(y[0])
# print(dummy_y[0])
# prediction = np.squeeze(prediction)
# prediction = np.squeeze(scaler.inverse_transform(prediction.reshape(-1,1)))
# prediction = [int(i) for i in prediction]

[9.0530841e-03 9.2716036e-06 6.5107219e-02 4.1214826e-06 2.0007104e-08
 4.6541858e-03 4.0769181e-13 3.0247653e-02 4.3111627e-13 5.4719009e-02
 4.1676693e-11 3.8411154e-03 2.8229496e-03 1.8723156e-01 2.7450874e-01
 1.5678731e-01 5.8001142e-11 1.4245415e-14 1.8642692e-01 2.7906929e-21
 3.5263679e-28 2.4586808e-02 3.1105180e-10 2.1993673e-08]
{68, 44, 49, 52, 56, 58, 59, 60, 61, 63}
{35, 36, 37, 38, 40, 42, 43, 44, 45, 47, 48, 49, 50, 52, 57, 59, 60, 61, 62, 63, 64, 66, 67, 69, 71, 72}


In [14]:
mid = MidiFile()
track = MidiTrack()
t = 0
print(notes)
for note in final_labels:
    # 147 means note_on
    # 67 is velosity
    note = np.asarray([147, note, 67])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    t += 1
    msg.time = t
    track.append(msg)
mid.tracks.append(track)
mid.save('ahora_quien_500_epochs_trained_on_bamboleo_4_notes_mystery_with_small_lr_shittonofnotes.mid')


test_mid = MidiFile()
test_track = MidiTrack()
t = 0

for note in test_notes:
    # 147 means note_on
    # 67 is velosity
    note = np.asarray([147, note, 67])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    t += 1
    msg.time = t
    test_track.append(msg)
test_mid.tracks.append(test_track)
test_mid.save('training_song.mid')


[37, 49, 52, 59, 52, 49, 59, 52, 49, 58, 37, 52, 49, 58, 37, 49, 52, 59, 52, 49, 59, 52, 49, 58, 37, 58, 52, 49, 37, 52, 49, 59, 49, 52, 59, 52, 49, 58, 37, 58, 49, 52, 37, 49, 52, 59, 52, 49, 59, 49, 52, 58, 49, 52, 58, 52, 49, 58, 49, 52, 58, 49, 52, 58, 52, 58, 49, 37, 37, 52, 49, 59, 59, 49, 52, 49, 58, 52, 37, 49, 58, 52, 37, 52, 49, 59, 49, 52, 59, 49, 58, 52, 49, 52, 58, 52, 49, 58, 49, 52, 58, 49, 52, 58, 52, 49, 58, 37, 42, 61, 57, 64, 61, 64, 57, 57, 61, 63, 42, 42, 63, 61, 57, 57, 61, 64, 64, 57, 61, 57, 61, 63, 61, 57, 63, 57, 61, 63, 61, 57, 63, 61, 57, 63, 61, 63, 57, 42, 42, 61, 57, 64, 61, 57, 64, 61, 57, 63, 42, 42, 61, 63, 57, 57, 61, 64, 57, 61, 64, 61, 57, 63, 61, 63, 57, 61, 57, 63, 61, 57, 63, 61, 57, 63, 61, 63, 57, 42, 37, 52, 59, 56, 56, 52, 59, 56, 52, 58, 37, 56, 58, 37, 52, 56, 52, 59, 56, 52, 59, 56, 52, 58, 56, 58, 52, 52, 56, 58, 56, 58, 52, 52, 56, 58, 58, 56, 52, 37, 56, 49, 68, 56, 49, 64, 68, 64, 61, 61, 68, 68, 66, 66, 63, 63, 59, 66, 59, 66, 45, 52,